<figure>
    <center> <img src="./Images/dd.png"><center/>
</figure>

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import difflib 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.model_selection import train_test_split

## Data Collection And Preprocessing

In [2]:
# Load the dataset
movie_data = pd.read_csv("./Datasets/movies.csv")
movie_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
# shape of the dataset
movie_data.shape

(4803, 24)

In [4]:
# selecting the relavent features for recommendation
selected_features =  ["genres", "keywords", "tagline", "cast", "director"]
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [5]:
# check missing values
movie_data.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [6]:
# replacing the null values with null string
for feature in selected_features:
    movie_data[feature] = movie_data[feature].fillna("")

In [7]:
# combine all the 5 selected features
combined_features = movie_data["genres"]+" "+movie_data["keywords"]+" "+movie_data["tagline"]+" "+movie_data["cast"]+" "+movie_data["director"]

In [8]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


### Convert The Text Feature Data To Vectors

In [9]:
vectorizer = TfidfVectorizer()

In [10]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [11]:
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

### Cosine Similarity

In [12]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [13]:
print(similarity) # can get how much similar the movie with other movies

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [14]:
similarity.shape

(4803, 4803)

In [54]:
# getting the movie name from the user
movie_name = input("Enter your favourite movie name: ")

Enter your favourite movie name: inception


In [55]:
# creating a list with all the movie names given in the dataset
list_of_all_titles = movie_data["title"].tolist()
list_of_all_titles

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [56]:
# finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
find_close_match

['Inception', 'Minions', 'Lincoln']

In [57]:
# get closest one
close_match = find_close_match[0]
close_match

'Inception'

In [58]:
# finding the index of the movie with title
index_of_the_movie = movie_data[movie_data.title == close_match]["index"].values[0]
index_of_the_movie

96

In [59]:
# getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))  # enumerate used for make loop in the list
similarity_score

[(0, 0.061940194404591926),
 (1, 0.07418756559461719),
 (2, 0.02356734374646841),
 (3, 0.19782094170329564),
 (4, 0.034687371115420196),
 (5, 0.042020268508437),
 (6, 0.008196478989478487),
 (7, 0.035956345517584115),
 (8, 0.042769381762091586),
 (9, 0.014048300475887631),
 (10, 0.051337877743053775),
 (11, 0.02555732250590339),
 (12, 0.024881206354169853),
 (13, 0.022370017376257496),
 (14, 0.04534559318744348),
 (15, 0.007731599344634815),
 (16, 0.03409425878909893),
 (17, 0.013914374148412966),
 (18, 0.03587125961705949),
 (19, 0.07436131105070441),
 (20, 0.06275407548243353),
 (21, 0.011955490731806677),
 (22, 0.06284288572966412),
 (23, 0.024331953034344656),
 (24, 0.04772080576312835),
 (25, 0.08817670269624489),
 (26, 0.037110075564865945),
 (27, 0.0820998091695857),
 (28, 0.06150775403038506),
 (29, 0.03946564929329973),
 (30, 0.0472075685110499),
 (31, 0.05470314570510593),
 (32, 0.007492127989734081),
 (33, 0.04148562546168602),
 (34, 0.0),
 (35, 0.048169877616417736),
 (36, 

In [60]:
# sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score, key=lambda x:x[1], reverse=True)   # x = similarity_score
sorted_similar_movies

[(96, 1.0000000000000002),
 (305, 0.2013538120541634),
 (3, 0.19782094170329564),
 (156, 0.1833867491298306),
 (3530, 0.18081466709336913),
 (3390, 0.17690879654663352),
 (1651, 0.16562023716440252),
 (1568, 0.16073119604474356),
 (2592, 0.16033179474097345),
 (176, 0.15796487085928348),
 (4401, 0.15392558635776168),
 (448, 0.15196643390373835),
 (1193, 0.15126702728529118),
 (76, 0.148642399975877),
 (883, 0.1479475485564351),
 (1380, 0.1465214146960398),
 (1028, 0.1462547305233708),
 (1452, 0.14559183221261673),
 (2514, 0.14169083686948142),
 (669, 0.14165693674581462),
 (297, 0.13435538946390166),
 (2152, 0.13311157589428138),
 (1107, 0.13227194154620892),
 (2757, 0.13093145062503636),
 (2371, 0.12730649998986743),
 (2091, 0.12695930379811957),
 (3431, 0.1249796210990898),
 (1431, 0.12489132606133929),
 (4664, 0.12269485609073193),
 (439, 0.12218656474802253),
 (1668, 0.12143217098713312),
 (1577, 0.12054704216690798),
 (4757, 0.11875411733950694),
 (1409, 0.11703245953285914),
 (12

In [61]:
# print the name of similar movies based on the index
print("Movies suggested for you: \n")

i=1
for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movie_data[movie_data.index==index]["title"].values[0]
    
    if (i<31):
        print(i, "-", title_from_index)
        i+=1

Movies suggested for you: 

1 - Inception
2 - Treasure Planet
3 - The Dark Knight Rises
4 - The Last Samurai
5 - Don Jon
6 - Hesher
7 - In Dreams
8 - Looper
9 - Highlander: Endgame
10 - The Revenant
11 - The Helix... Loaded
12 - Cold Mountain
13 - The Count of Monte Cristo
14 - G.I. Joe: The Rise of Cobra
15 - Catch Me If You Can
16 - The Man in the Iron Mask
17 - Solaris
18 - The Walk
19 - Top Gun
20 - Sin City: A Dame to Kill For
21 - Blood Diamond
22 - Eternal Sunshine of the Spotless Mind
23 - The Juror
24 - Letters from Iwo Jima
25 - RockNRolla
26 - The Silence of the Lambs
27 - Layer Cake
28 - Premium Rush
29 - Bronson
30 - Shutter Island


## Movie Recomendation System

In [43]:
movie_detail = input("Enter your favourite movie name: ")

list_of_all_titles = movie_data["title"].tolist()
find_close_match = difflib.get_close_matches(movie_detail, list_of_all_titles)

if find_close_match:
    # Retrieve the first close match
    close_match = find_close_match[0]
    
    index_of_the_movie = movie_data[movie_data.title == close_match]["index"].values[0]
    similarity_score = list(enumerate(similarity[index_of_the_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x:x[1], reverse=True)
    
    # print the name of similar movies based on the index
    print("Movies suggested for you: \n")
    
    i=1
    for movie in sorted_similar_movies:
        index = movie[0]
        title_from_index = movie_data[movie_data.index==index]["title"].values[0]
        
        if (i<31):
            print(i, "-", title_from_index)
            i+=1
else:
    print("No close match movies found.")

Enter your favourite movie name: avatar
Movies suggested for you: 

1 - Avatar
2 - Alien
3 - Aliens
4 - Guardians of the Galaxy
5 - Star Trek Beyond
6 - Star Trek Into Darkness
7 - Galaxy Quest
8 - Alien³
9 - Cargo
10 - Trekkies
11 - Gravity
12 - Moonraker
13 - Jason X
14 - Pocahontas
15 - Space Cowboys
16 - The Helix... Loaded
17 - Lockout
18 - Event Horizon
19 - Space Dogs
20 - Machete Kills
21 - Gettysburg
22 - Clash of the Titans
23 - Star Wars: Clone Wars: Volume 1
24 - The Right Stuff
25 - Terminator Salvation
26 - The Astronaut's Wife
27 - Planet of the Apes
28 - Star Trek
29 - Wing Commander
30 - Sunshine


## Movie Recomendation System For Any Movie/ Cast/ Director Name

In [62]:
def get_recommendations(input_detail):
    # creating a list with all the movie names given in the dataset
    list_of_all_titles = movie_data["title"].tolist()
    list_of_all_cast = movie_data["cast"].tolist()
    list_of_all_directors = movie_data["director"].tolist()

    # finding the close match for the input given by the user
    find_close_match_title = difflib.get_close_matches(input_detail, list_of_all_titles)
    find_close_match_cast = difflib.get_close_matches(input_detail, list_of_all_cast)
    find_close_match_director = difflib.get_close_matches(input_detail, list_of_all_directors)

    if find_close_match_title:
        # If the input matches a movie title
        close_match = find_close_match_title[0]
        index_of_the_movie = movie_data[movie_data.title == close_match]["index"].values[0]
    elif find_close_match_cast:
        # If the input matches a cast name
        close_match = find_close_match_cast[0]
        index_of_the_movie = movie_data[movie_data.cast.str.contains(close_match)]["index"].values[0]
    elif find_close_match_director:
        # If the input matches a director name
        close_match = find_close_match_director[0]
        index_of_the_movie = movie_data[movie_data.director.str.contains(close_match)]["index"].values[0]
    else:
        print("No close match found.")
        return
    
    # getting a list of similar movies
    similarity_score = list(enumerate(similarity[index_of_the_movie]))  # enumerate used for making a loop in the list
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)   # x = similarity_score

    # print the name of similar movies based on the index
    print("Movies suggested for you: \n")
    i = 1
    for movie in sorted_similar_movies:
        index = movie[0]
        title_from_index = movie_data[movie_data.index == index]["title"].values[0]
        if i < 31:
            print(i, "-", title_from_index)
            i += 1

# getting the input from the user
input_detail = input("Enter your favourite movie name, cast or director: ")
get_recommendations(input_detail)

Enter your favourite movie name, cast or director: Chistopher nolan
Movies suggested for you: 

1 - The Dark Knight Rises
2 - Batman Begins
3 - The Dark Knight
4 - The Prestige
5 - Batman Returns
6 - Interstellar
7 - Batman & Robin
8 - Batman
9 - Inception
10 - Superman
11 - Legend
12 - Kick-Ass 2
13 - Kick-Ass
14 - Child 44
15 - Get Smart
16 - The Killer Inside Me
17 - RockNRolla
18 - Watchmen
19 - Suicide Squad
20 - True Romance
21 - Bronson
22 - The Other Side of Heaven
23 - Superman II
24 - Superman III
25 - Jonah Hex
26 - Batman Forever
27 - Insomnia
28 - Captain Corelli's Mandolin
29 - Catwoman
30 - The Contender
